# OC20 metrics on MD17
This notebook presents the improvement of bias appended model and roiginal model which are trained on the 8 molecule in MD17.
performances are evaluated on metric: Energy MAE, Force MAE, EFwT defined by OC20.
Link to MD17 and OC20 can be found in README. 

please download the pre-trained model linked in README and put them in checkpoints directory.

In [3]:
import sys
sys.path.append("../../")

import csv
import dill
import torch
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader

from datasets.MD17.MD17Dataset import MD17SingleDataset
from scripts.losses import EnergyLoss, PosForceLoss
from scripts.util import *

In [4]:
def get_configs(model_path):
    configs = dict()
    with open(f"{model_path}/info.txt") as fp:
        for line in fp:
            line = line.strip().split(" ")
            if len(line)>1:
                configs[line[0]] = line[1]

    return configs

def predict(model_path, epoch, configs, device, root):
    # initialize
    dataset = MD17SingleDataset(configs["style"], configs["molecule"], "test", configs["split"], root)
    identifier = dataset.identifier
    test_dataloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=dataset.collate)
    
    path = glob.glob(f"{model_path}/{epoch:03d}_*.pth")
    if not path: raise ValueError("model not found")
    model = torch.load(path[0], map_location=torch.device(torch.cuda.current_device()), pickle_module=dill)
    print(f"Using model {path[0]}")
    model.eval()
    
    # test
    preds = []
    losses = []
    tq = tqdm(test_dataloader)
    for data, label in tq:
        data = {i:v.to(device) for i, v in data.items()}
        label = {i:v.to(device) for i, v in label.items()}
        pred = model(data)
        preds.append( ((torch.cat((pred["E"], pred["F"].reshape(1, -1)), axis=1)).squeeze()).tolist() )
     
        loss_E = (EnergyLoss(pred, label).to("cpu").item())
        loss_F = PosForceLoss(pred, label)
        loss_F = [ (l.to("cpu").item()) for l in loss_F]
        losses.append([loss_E]+loss_F)
    
    # save result
    save_reult(model_path, identifier, preds, losses)

def metric(model_path, configs):
    # Every dataset has different unit, unify them to Energy:eV and Force: eV/A
    toEv = {"MD17SingleDataset":0.0433634, "MD17Dataset":0.0433634}
    scale = toEv[configs["dataset"]]

    # load and calculate
    style, molecule, split = configs["style"], configs["molecule"], configs["split"]
    with open(f"{model_path}/loss_MD17SingleDataset_{style}_{molecule}{split}test.csv", newline='') as fp:
        cdata = list(csv.reader(fp, quoting=csv.QUOTE_NONNUMERIC))
        cdata = [[c for c in row] for row in cdata]
        loss_Emole , loss_Fmole= [], []
        hit, total = 0, len(cdata)
        failE, failF = 0, 0

        for row in tqdm(cdata):
            loss_Emole.append(row[0]) 
            loss_Fmole.append(row[1:])
            m = max(row[1:])
            if row[0]<=0.02/scale and m<=0.03/scale: hit += 1
            else:
                if row[0]>0.02/scale: failE += 1
                if m>0.03/scale: failF += 1
        loss_Emole , loss_Fmole= np.array(loss_Emole), np.array(loss_Fmole)

        print(f"Energy MAE: {np.mean(loss_Emole):.3f},\tForce MAE: {np.mean(loss_Fmole):.3f},\tEFwT: {hit/total:.3f} (failE:{(failE/total):.3f}, failF:{(failF/total):.3f})\n")

# Perofrmance of original Schnet

In [6]:
# turn this on if you want to re-predict
need_predict = False

for m in ["a", "b", "e", "m", "n", "s", "t", "u"]:
    print(f"molecule:{m}")
    model_path = f"./checkpoints/BGR-schnet(no-bias-{m})"
    cfg = get_configs(model_path)

    if need_predict: predict(model_path, 299, cfg, "cuda", "../../datasets/MD17/datas")
    metric(model_path, cfg)

molecule:a


100%|██████████| 156762/156762 [00:00<00:00, 190316.76it/s]


Energy MAE: 242.458,	Force MAE: 0.442,	EFwT: 0.000 (failE:1.000, failF:1.000)

molecule:b


100%|██████████| 572983/572983 [00:01<00:00, 322508.71it/s]


Energy MAE: 14.520,	Force MAE: 0.276,	EFwT: 0.000 (failE:1.000, failF:0.713)

molecule:e


...

Energy MAE: 197.106,	Force MAE: 0.314,	EFwT: 0.000 (failE:1.000, failF:0.990)

molecule:t


100%|██████████| 387790/387790 [00:01<00:00, 362305.79it/s]


Energy MAE: 118.788,	Force MAE: 0.304,	EFwT: 0.000 (failE:1.000, failF:0.958)

molecule:u


100%|██████████| 78770/78770 [00:00<00:00, 296519.00it/s]


Energy MAE: 440.164,	Force MAE: 0.376,	EFwT: 0.000 (failE:1.000, failF:0.999)



# Performance on bias appended Schnet

In [ ]:
# turn this on if you want to re-predict
need_predict = False

for m in ["a", "b", "e", "m", "n", "s", "t", "u"]:
    print(f"molecule:{m}")
    model_path = f"./checkpoints/BGR-schnet(biased-{m})"
    cfg = get_configs(model_path)

    if need_predict: predict(model_path, 299, cfg, "cuda", "../../datasets/MD17/datas")
    metric(model_path, cfg)